# Test GPU+CUDA

In [1]:
import tensorflow as tf
tf.__version__

'1.8.0'

In [2]:
a = tf.constant([1.0, 2.0, 3.0 ,4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0 ,4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8751735475784606111
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1436214886
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4884826021282177807
physical_device_desc: "device: 0, name: GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [4]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from __future__ import print_function
from model.preprocessing_helper import CANVAS_SIZE, EMBEDDING_DIM
from model.unet import UNet


config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Infer

In [5]:
# -*- coding: utf-8 -*-
import os
from model.utils import compile_frames_to_gif

In [6]:
################ infer-params ###############

model_dir='experiments_finetune/checkpoint/experiment_0'  # directory that saves the model checkpoints
batch_size=32       # directory that saves the model checkpoints
source_obj='experiments_finetune/data/val.obj'   # the source images for inference
embedding_ids='67'     # embeddings involved
save_dir='save_dir/'  # path to save inferred images
inst_norm=0         # use conditional instance normalization in your model
interpolate=0       # interpolate between different embedding vectors
steps=10            # interpolation steps in between vectors
output_gif='0'        # output name transition gif
uroboros=0          # Shōnen yo, you have stepped into uncharted territory

In [7]:


with tf.Session(config=config) as sess:
    model = UNet(batch_size=batch_size)
    model.register_session(sess)
    model.build_model(is_training=False, inst_norm=inst_norm)
    embedding_ids = [int(i) for i in embedding_ids.split(",")]
    if not interpolate:
        if len(embedding_ids) == 1:
            embedding_ids = embedding_ids[0]
        model.infer(model_dir=model_dir, source_obj=source_obj, embedding_ids=embedding_ids,
                    save_dir=save_dir)
    else:
        if len(embedding_ids) < 2:
            raise Exception("no need to interpolate yourself unless you are a narcissist")
        chains = embedding_ids[:]
        if uroboros:
            chains.append(chains[0])
        pairs = list()
        for i in range(len(chains) - 1):
            pairs.append((chains[i], chains[i + 1]))
        for s, e in pairs:
            model.interpolate(model_dir=model_dir, source_obj=source_obj, between=[s, e],
                              save_dir=save_dir, steps=steps)
        if output_gif:
            gif_path = os.path.join(save_dir, output_gif)
            compile_frames_to_gif(save_dir, gif_path)
            print("gif saved at %s" % gif_path)




unpickled total 10 examples
examples -> 10

model_checkpoint_path: "experiments_finetune/checkpoint/experiment_0\\unet.model-0"
all_model_checkpoint_paths: "experiments_finetune/checkpoint/experiment_0\\unet.model-0"


INFO:tensorflow:Restoring parameters from experiments_finetune/checkpoint/experiment_0\unet.model-0


NotFoundError: Key generator/g_d1_bn/beta not found in checkpoint
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]
	 [[Node: save/RestoreV2/_79 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_84_save/RestoreV2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'save/RestoreV2', defined at:
  File "C:\ProgramData\Anaconda3\envs\gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\gpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\gpu\lib\